In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\snetkova\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\snetkova\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.manifold import TSNE

In [3]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

### get files

In [4]:
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive

# gauth = GoogleAuth()
# drive = GoogleDrive(gauth)

In [5]:
# downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
# downloaded.GetContentFile('cc.ru.300.vec.zip')

In [6]:
# downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
# downloaded.GetContentFile('cc.uk.300.vec.zip')

In [7]:
# import zipfile
# with zipfile.ZipFile('cc.ru.300.vec.zip', 'r') as zip_ref:
#     zip_ref.extractall('')

In [8]:
# with zipfile.ZipFile('cc.uk.300.vec.zip', 'r') as zip_ref:
#     zip_ref.extractall('')

### mapping

In [9]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")

In [10]:
uk_emb.most_similar([uk_emb["iснує"]])

[('iснує', 1.0),
 ('iснують', 0.722436785697937),
 ('iснувати', 0.6935151219367981),
 ('iснували', 0.6692912578582764),
 ('Iснує', 0.6570639610290527),
 ('розумiють', 0.6478244066238403),
 ('Оскiльки', 0.6471762657165527),
 ('зберiгається', 0.6427539587020874),
 ('взагалi', 0.6312386393547058),
 ('зрозумiло', 0.6258929967880249)]

In [11]:
ru_emb.most_similar([uk_emb["iснує"]])

[('СРН', 0.2498701810836792),
 ('Shadowcaster', 0.23073866963386536),
 ('эсеровской', 0.23030570149421692),
 ('Гегечкори', 0.22700092196464539),
 ('СДПР', 0.2228643000125885),
 ('инициативе', 0.22138282656669617),
 ('страницыКомментарии', 0.22029446065425873),
 ('1918г.', 0.21931448578834534),
 ('Аушев', 0.21862442791461945),
 ('законодательной', 0.21654823422431946)]

In [12]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t") 
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru)) # пара из словаря UA -> RU
            uk_vectors.append(uk_emb[uk])  
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [13]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

In [14]:
uk_ru_train[0]

('iснує', 'существует')

In [15]:
from sklearn.linear_model import LinearRegression
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

In [16]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
september = mapping.predict(uk_emb["вересень"].reshape(1, -1))
march = mapping.predict(uk_emb["березень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8541286587715149),
 ('июнь', 0.8411203026771545),
 ('март', 0.839699387550354),
 ('сентябрь', 0.8359869122505188),
 ('февраль', 0.832929790019989),
 ('октябрь', 0.8311846256256104),
 ('ноябрь', 0.8278924226760864),
 ('июль', 0.8234529495239258),
 ('август', 0.8120501041412354),
 ('декабрь', 0.803900420665741)]

In [17]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == mapped_vectors.shape[0]
    num_matches = 0
   
    for i, (_, ru) in enumerate(pairs):
        top_words = []
        for word in ru_emb.most_similar([mapped_vectors[i]])[: topn]:
            top_words.append(word[0])
#         print(ru, top_words)
        if np.isin(ru, top_words).sum():
            num_matches +=1
#     print(num_matches)
        
    precision_val = num_matches / len(pairs)
    return precision_val

In [18]:
precision([("серпень", "август")], august, topn=5)

0.0

In [19]:
precision([("серпень", "август"), ("вересень", "сентябрь"), ("березень", "март")], np.vstack((august, september, march)), topn=3)

0.6666666666666666

In [20]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [21]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [22]:
precision(uk_ru_test, mapping.predict(X_test), 1), precision(uk_ru_test, mapping.predict(X_test), 5)

(0.6356589147286822, 0.813953488372093)

In [23]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    data = np.matmul(X_train.T, Y_train)
    U, s, vh = np.linalg.svd(data)
    return np.matmul(U, vh)

In [24]:
W = learn_transform(X_train, Y_train)

In [25]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])[0][0]

'апрель'

In [26]:
precision(uk_ru_test, np.matmul(X_test, W)), precision(uk_ru_test, np.matmul(X_test, W), 5)

(0.6537467700258398, 0.8242894056847545)

In [27]:
ru_emb.most_similar([np.matmul(uk_emb["зловив"], W)])[0][0]

'поймал'

### word by word translator

In [28]:
import io
f = io.open("fairy_tale.txt", mode="r", encoding="utf-8")
uk_sentences = [line.rstrip().lower() for line in f]

In [29]:
uk_sentences[0]

'лисичка - сестричка і вовк - панібрат'

In [30]:
# uk_emb['панібрат']

In [31]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = []
    for word in sentence.split(' '):
        try:
            translated.append(ru_emb.most_similar([np.matmul(uk_emb[word], W)])[0][0])
        except KeyError as e:
            translated.append(word)
            
    translation = ' '.join(translated)
            
    return translation

In [32]:
translate(uk_sentences[0])

'лисичка – сестричка и волк – панібрат'

In [33]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"

In [34]:
translate("кіт зловив мишу")

'кот поймал мышку'

In [35]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

src: лисичка - сестричка і вовк - панібрат
dst: лисичка – сестричка и волк – панібрат

src: як була собі лисичка , да й пішла раз до однії баби добувать огню ; ввійшла у хату да й каже : " добрий день тобі , бабусю !
dst: как была себе лисичка , че и пошла раз к однії бабы добувать огня ; вошла во избу че и говорит : " хороший день тебе , бабушку !

src: дай мені огня " .
dst: дай мне огня " .

src: а баба тільки що вийняла із печі пирожок із маком , солодкий , да й положила , щоб він прохолов ; а лисичка се і підгледала , да тілько що баба нахилилась у піч , щоб достать огня , то лисичка зараз ухватила пирожок да і драла з хати , да , біжучи , весь мак із його виїла , а туда сміття наклала .
dst: а бабка только что вынула со печи пирожок со маком , сладкий , че и согнула , чтобы он прохолов ; а лисичка ой и підгледала , че токмо что бабка качнулась во печь , чтобы достать огня , то лисичка сейчас ухватила пирожок че и деру со хаты , че , пробежать , весь мак со его виїла , а туда мусо

src: де се ти набрала стільки риби ? "
dst: куда ой ты набрала столько рыбы ? "

src: вона каже : " наловила , вовчику - братику ! "
dst: она говорит : " наловила , вовчику – братику ! "

src: а собі на думці : " подожди , і я зроблю з тобою таку штуку , як і ти зо мною " .
dst: а себе по мнении : " подожди , и мной сделаю со тобой такую штуку , как и ты За мной " .

src: - " як же ти ловила ? "
dst: – " как то ты ловила ? "

src: - " так , вовчику , уложила хвостик в ополонку , вожу тихенько да й кажу ; ловися , рибка , мала і велика !
dst: – " так , вовчику , уложила хвостик во прорубь , вожу тихонько че и говорю ; ловися , рыбка , имела и большая !

src: коли хочеш , то і ти піди , налови собі " .
dst: когда хочешь , то и ты пойди , налови себе " .

src: він побіг да зробив так , як казала лисичка .
dst: он побежал че сделал так , как говорила лисичка .

src: а лисичка стала за деревом да й дивиться ; коли у вовчика зовсім хвостик примерз , вона тоді побігла в село да й кричить : " 